# Reranking
* The reranking stage happens after the ranking stage
* The ranking stage learns the user's preference relation over items
  * Does the user prefer item A over item B?
* The reranking stage learns the user's preference relation over recommendation lists
  * Does the user prefer recommendation list A over recommendation list B? 
* The reranking stage is modelled as a binary quadratic programming problem
  * Given a set of items, find the optimal subset by solving a QP
  * We get a ranked list by iteratively taking subsets

In [ ]:
using JuMP, SCIP

In [ ]:
@memoize function get_similarity_metric(medium::String, weights = nothing)
    alphas = ["$medium/all/WatchSimilarity", "$medium/all/GenreSimilarity"]
    if medium == "anime"
        push!(alphas, "$medium/all/TagSimilarity")
    end
    if isnothing(weights)
        weights = ones(length(alphas))
    end
    weights = convert.(Float16, weights ./ sum(weights))
    
    S = zeros(Float16, num_items(medium), num_items(medium))
    for (a, w) in Iterators.zip(alphas, weights)
        GC.gc()
        S += read_params(a)["S"] * w
    end
    return S
end

In [ ]:
function solve_linear_program(list_size, relevance_scores, constraint_spec)
    # solves the linear program:
    # maximize relevance_scores * x 
    # with the constraints:
    # sum(x) = list_size
    # x \in {0, 1} are binary variables    
    # attributes[i]' * x <= constraints[i] for all i

    if list_size == 0
        return []
    end

    # scale problem by list_size
    N = length(relevance_scores)
    relevance_scores = relevance_scores ./ list_size
    attributes = constraint_spec[1]
    constraints = [f(list_size) for f in constraint_spec[2]]

    # setup optimization problem in JuMP
    model = Model(SCIP.Optimizer)
    set_attribute(model, "display/verblevel", 0)
    @variable(model, 0 <= x[1:N] <= 1, Bin)
    @constraint(model, sum(x) == list_size)
    for i = 1:length(constraints)
        @constraint(model, attributes[i]' * x <= constraints[i])
    end
    @objective(model, Max, relevance_scores' * x)

    # solve
    optimize!(model)
    try
        solution = value.(x)
        return sortperm(solution)[end-list_size+1:end]
    catch
        # no feasible points. Randomly drop a consraint and retry
        @assert length(constraints) > 0
        skip = (list_size % length(constraints)) + 1
        return solve_linear_program(
            list_size,
            relevance_scores,
            (
                [v for (i, v) in Iterators.enumerate(constraint_spec[1]) if i != skip],
                [v for (i, v) in Iterators.enumerate(constraint_spec[2]) if i != skip],
            ),
        )
    end
    @assert false
end;

In [ ]:
function solve_quadratic_program(
    list_size,
    similarity_metric,
    relevance_scores,
    constraint_spec,
)
    # solves the quadratic program:
    # minimize x' * similarity_metric * x - relevance_scores * x 
    # with the constraints:
    # sum(x) = list_size
    # x \in {0, 1} are binary variables    
    # attributes[i]' * x <= constraints[i] for all i

    if list_size == 0
        return []
    end

    # scale problem by list_size
    N = size(similarity_metric)[1]
    similarity_metric = similarity_metric ./ list_size^2
    relevance_scores = relevance_scores ./ list_size
    attributes = constraint_spec[1]
    constraints = [f(list_size) for f in constraint_spec[2]]

    # solve the mixed-integer quadratic problem by doing a brute 
    # force search. 
    @assert N - 1 == list_size # limit N for performance
    scores = zeros(N)
    for i = 1:N
        x = ones(N)
        x[i] = 0
        feasible = true
        for (a, c) in Iterators.zip(attributes, constraints)
            if a' * x > c
                feasible = false
                break
            end
        end
        if feasible
            scores[i] = x' * similarity_metric * x - relevance_scores' * x
        else
            scores[i] = Inf
        end
    end

    # if no feasible points are found, drop a random constraint and retry
    if all(scores .== Inf)
        @assert length(constraints) > 0
        skip = (list_size % length(constraints)) + 1
        return solve_quadratic_program(
            list_size,
            similarity_metric,
            relevance_scores,
            (
                [v for (i, v) in Iterators.enumerate(constraint_spec[1]) if i != skip],
                [v for (i, v) in Iterators.enumerate(constraint_spec[2]) if i != skip],
            ),
        )
    end

    chosen = argmin(scores)
    [x for x in 1:N if x != chosen]
end;

In [ ]:
function get_seasonal_constraints(df, limitfrac)
    attributes = []
    constraints = []
    start_seasons = copy(df.start_season)
    start_seasons[ismissing.(start_seasons)] .= ""
    seasons = Set(start_seasons)
    for season in Set(start_seasons)
        if season == ""
            continue
        end
        attrib = convert.(Float32, start_seasons .== season)
        if sum(attrib) > 1
            push!(attributes, attrib)
            push!(constraints, N -> ceil(limitfrac * N))
        end
    end
    attributes, constraints
end

function get_intrarelated_constraints(df, medium, limitfrac)
    S = read_params("$medium/all/RelatedSeries/similarity_matrix")["S"]
    attributes = []
    constraints = []
    for uid in df.uid
        x = S[:, uid][df.uid]
        if sum(x) > 1
            if any(all(x .== y) for y in attributes)
                continue
            end
            push!(attributes, collect(x))
            push!(constraints, N -> 1 + limitfrac * N)
        end
    end
    attributes, constraints
end

function get_interrelated_constraints(df, limitfrac)
    # num items related to the same series
    attributes = [convert.(Float32, df.is_related .!= 0)]
    constraints = [N -> ceil(limitfrac * N)]
    attributes, constraints
end

function get_crossrelated_constraints(df, limitfrac)
    # num items related to any watched series 
    attributes = [convert.(Float32, df.is_cross_related .!= 0)]
    constraints = [N -> ceil(limitfrac * N)]
    attributes, constraints
end

function get_ptw_constraints(df, limitfrac)
    # num items related to any watched series     
    attributes = [convert.(Float32, df.ptw .!= 0)]
    constraints = [N -> ceil(limitfrac * N)]
    attributes, constraints
end

function merge_constraints(args...)
    Tuple(reduce(vcat, [x[t] for x in args]) for t = 1:2)
end

function get_constraint_spec(
    df,
    medium,
    spec,
)
    constraints = [
        get_intrarelated_constraints(df, medium, spec[:intrarelated]),
        get_interrelated_constraints(df, spec[:interrelated]),
        get_crossrelated_constraints(df, spec[:crossrelated]),
        get_ptw_constraints(df, spec[:ptw]),
    ]
    if medium == "anime"
        push!(constraints, get_seasonal_constraints(df, spec[:seasonal]))
    end
    merge_constraints(constraints...)
end;

In [ ]:
function rerank(df, medium::String, list_size, similarity_spec, constraint_spec)
    S =
        similarity_spec[:penalty] *
        get_similarity_metric(medium, similarity_spec[:weights])[df.uid, df.uid]
    constraint_spec = get_constraint_spec(df, medium, constraint_spec)
    candidates = solve_linear_program(list_size, df.score, constraint_spec)

    order = []
    while length(candidates) > 0
        new_S = S[candidates, candidates]
        new_score = df.score[candidates]
        new_constraint_spec =
            ([x[candidates] for x in constraint_spec[1]], constraint_spec[2])
        new_order = solve_quadratic_program(
            length(candidates) - 1,
            new_S,
            new_score,
            new_constraint_spec,
        )
        worst_candidate = candidates[findfirst(x -> x ∉ new_order, 1:length(candidates))]
        candidates = candidates[new_order]
        pushfirst!(order, worst_candidate)
    end

    df[order, :]
end
rerank(args...) = x -> rerank(x, args...);